# Correcting enumerator assignments

In [1]:
# Set up
import pandas as pd
import numpy as np
import sys
from pathlib import Path
CODE_ROOT = Path.cwd().parents[1]
sys.path.append(str(CODE_ROOT))
import config
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill
import os

In [2]:
# Load datasets
existing_assignments = pd.read_csv(config.ENUMERATORS / "assignedlabs.csv")
all_enumerators = pd.read_csv(config.ENUMERATORS / "all_enumerators.csv")
assignment_corrections = pd.read_excel(config.ENUMERATORS / "enumerator_corrections.xlsx")

In [3]:
# Correct assignments
for _, row in assignment_corrections.iterrows():
    enum_id_old = row["enum_id_old"]
    enum_id_new = row["enum_id_new"]
    labgroupid = row["labgroupid"]

    # Extract the new enumerator details from all_enumerators
    new_enum_details = all_enumerators[all_enumerators["enum_id"] == enum_id_new].iloc[0]

    # Find the lab group in existing_assignments
    mask = existing_assignments["labgroupid"] == labgroupid

    # Check if lab group enumerator already updated
    already_updated = (
        existing_assignments.loc[mask, "enum_id"] == enum_id_new
    ).any()
    
    # Update the existing assignments if not already updated
    if already_updated:
        continue
    else:
        # replace the enumerator details (enum_id, enum_firstname, enum_lastname, enum_email)
        existing_assignments.loc[
            mask, ["enum_id", "enum_firstname", "enum_lastname", "enum_email"]] = [
            new_enum_details["enum_id"],
            new_enum_details["firstname"],
            new_enum_details["lastname"],
            new_enum_details["email_cleaned"]
        ]


In [4]:
# Save updated assignments
existing_assignments.to_csv(config.ENUMERATORS / "assignedlabs.csv", index=False)